In [78]:
import pandas as pd
from util.db_util import Database

def get_top_state_meet_returners(year, gender, event, limit=10):
    db = Database("db/Track.db")

    query = """
    SELECT
      ar.*,
      a.first,
      a.last,
      m.gender,
      m.year,
      s.school_name AS school
    FROM athlete_result ar
    JOIN meet m
      ON ar.meet_id = m.meet_id
    JOIN athlete a
      ON ar.athlete_id = a.athlete_id
    JOIN school s
      ON a.school_id = s.school_id
    """
    df = pd.read_sql(query, db.conn)
    
    filtered_df = df[df["event"] == event]
    filtered_df = filtered_df[filtered_df["gender"] == gender]
    filtered_df = filtered_df[filtered_df["year"] == year]
    filtered_df = filtered_df[filtered_df["grade"] != "SR"]
    
    all_field_events = ["Discus" , "Shot Put", "Long Jump", "High Jump", " Pole Vault"]
    if event in all_field_events:
        sorted_df = filtered_df.sort_values(by = "result2", ascending = False)
    else:
        sorted_df = filtered_df.sort_values(by = "result2", ascending = True)
        
    sorted_df = sorted_df.drop_duplicates(subset = "athlete_id")
    sorted_df = sorted_df.head(limit)
    
    sorted_df = sorted_df.reset_index(drop=True)
    sorted_df['place'] = sorted_df.index + 1

    columns_to_keep = ['place', 'first', 'last', 'school', 'result', 'grade']
    sorted_df = sorted_df[columns_to_keep]

    grad_map = {'JR': year + 1, 'SO': year + 2, 'FR': year + 3}
    sorted_df['grad year'] = sorted_df['grade'].map(grad_map)
    sorted_df = sorted_df.drop(columns=['grade'])
    
    return sorted_df

print(get_top_state_meet_returners(2025, "Boys", "High Jump"))

   place      first        last                     school    result  \
0      1     JORDAN     RANDALL           Warsaw Community  7' 1.25"   
1      2       NOAH        HALL                Owen Valley     6' 9"   
2      3     KONRAD      HAYDEN                    Fishers     6' 8"   
3      4     DUSTIN  RICHARDSON                 West Noble     6' 7"   
4      5    BRADLEY   STREVELER         Greenfield-Central     6' 6"   
5      6     HAYDEN       KORTE    Lakewood Park Christian     6' 6"   
6      7      ISAAC       STEAD                 Plainfield     6' 5"   
7      8  ALEXANDER          DO           Lawrence Central     6' 5"   
8      9        ELI     BERTSCH                   Bluffton     6' 5"   
9     10   BENJAMIN  MCBREAIRTY  Harrison (West Lafayette)     6' 5"   

   grad year  
0       2026  
1       2026  
2       2026  
3       2026  
4       2027  
5       2026  
6       2026  
7       2026  
8       2026  
9       2026  
